In [25]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import tempfile
import math

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

# machine learning
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

In [26]:
train_df = pd.read_csv('data/train.csv', converters={'Cabin': str, 'SibSp': str})
test_df = pd.read_csv('data/train.csv')

In [27]:
train_df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,,C


In [28]:
train_df['Cabin']==''

0       True
1      False
2       True
3      False
4       True
5       True
6      False
7       True
8       True
9       True
10     False
11     False
12      True
13      True
14      True
15      True
16      True
17      True
18      True
19      True
20      True
21     False
22      True
23     False
24      True
25      True
26      True
27     False
28      True
29      True
       ...  
861     True
862    False
863     True
864     True
865     True
866     True
867    False
868     True
869     True
870     True
871    False
872    False
873     True
874     True
875     True
876     True
877     True
878     True
879    False
880     True
881     True
882     True
883     True
884     True
885     True
886     True
887    False
888     True
889    False
890     True
Name: Cabin, dtype: bool

In [29]:
list(train_df.columns)

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [30]:
CONTINUOUS_COLUMNS = ['Pclass',
                      'Fare',
                      'Age',
                      'Parch'
                     ]

In [31]:
CATEGORICAL_COLUMNS = [
                       'Sex', 
                       'SibSp',
                       #'Cabin', 
                       'Embarked'
                      ]

In [32]:
##creating the feature tensor and labels

In [33]:

def input_fn(df):
    # Creates a dictionary mapping from each continuous feature column name (k) to
    # the values of that column stored in a constant Tensor.
    continuous_cols = {k: tf.constant(df[k].values)
                     for k in CONTINUOUS_COLUMNS}
    # Creates a dictionary mapping from each categorical feature column name (k)
    # to the values of that column stored in a tf.SparseTensor.
    categorical_cols = {k: tf.SparseTensor(indices=[[i, 0] for i in range(df[k].size)], values=df[k].values, dense_shape=[df[k].size, 1])
                      for k in CATEGORICAL_COLUMNS}
    # Merges the two dictionaries into one.
    feature_cols = dict(continuous_cols.items() | categorical_cols.items())
    # Converts the label column into a constant Tensor.
    label = tf.constant(df['Survived'].values)
    # Returns the feature columns and the label.
    return feature_cols, label


def train_input_fn():
    return input_fn(train_df)

def eval_input_fn():
    return input_fn(test_df)

In [34]:
## define the categorised values

In [35]:
Pclass= tf.contrib.layers.sparse_column_with_hash_bucket(column_name="Pclass", hash_bucket_size=10)
gender = tf.contrib.layers.sparse_column_with_keys(column_name="Sex", keys=["male", "female"])
SibSp=tf.contrib.layers.sparse_column_with_hash_bucket(column_name="SibSp", hash_bucket_size=10)
Embarked=tf.contrib.layers.sparse_column_with_hash_bucket(column_name="Embarked", hash_bucket_size=10)

In [36]:
model_dir = tempfile.mkdtemp()
m = tf.contrib.learn.LinearClassifier(feature_columns=[
   gender
    #SibSp
   # Embarked
],optimizer=tf.train.FtrlOptimizer(
    learning_rate=0.1,
    l1_regularization_strength=1.0,
    l2_regularization_strength=1.0),
  model_dir=model_dir)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x11408a240>, '_master': '', '_num_ps_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000}


In [37]:
m.fit(input_fn=train_input_fn, steps=10000)

TypeError: Expected binary or unicode string, got nan

In [15]:
results = m.evaluate(input_fn=eval_input_fn, steps=1)
for key in sorted(results):
    print ("%s: %s" % (key, results[key]))

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Starting evaluation at 2017-04-23-06:22:11
INFO:tensorflow:Evaluation [1/1]
INFO:tensorflow:Finished evaluation at 2017-04-23-06:22:11
INFO:tensorflow:Saving dict for global step 10000: accuracy = 0.786756, accuracy/baseline_label_mean = 0.383838, accuracy/threshold_0.500000_mean = 0.786756, auc = 0.766873, global_step = 10000, labels/actual_label_mean = 0.383838, labels/prediction_mean = 0.383839, loss = 0.515041, precision/positive_threshold_0.500000_mean = 0.742038, recall/positive_threshold_0.500000_mean = 0.681287
accuracy: 0.786756
accuracy/baseline_label_mean: 0.383838
accuracy/threshold_0.500000_mean: 0.786756
auc: 0.766873
global_step: 10